In [1]:
import pandas as pd
import numpy as np
import time
import datetime as dt
from datetime import datetime
from datetime import date, timedelta
import json

In [2]:
df = pd.concat ([pd.read_json(f,dtype = str) for f in
                 [
                     '2003_Tracking.json',
                    # '2003_Tracking001.json',                    
                     
                 ]], ignore_index = True)

### Olahan

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4222 entries, 0 to 4221
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   summary    4222 non-null   object
 1   histories  4222 non-null   object
dtypes: object(2)
memory usage: 66.1+ KB


In [4]:
summer = df.summary
hist = df.histories
hist_gabung = []
summer_gabung = []
eror = '________________________'


for j in range(len(summer)):
    sumer0 = summer.iloc[j]
    sumer0 = sumer0.replace('"', '')
    sumer0 = sumer0.replace("\\", "")     
    sumer0 = sumer0.replace("'", '"')
    sumer0 = json.loads(sumer0)
    sumer0 = pd.DataFrame([sumer0], columns = sumer0.keys())
    
    sumer0 = sumer0[['waybill_number','receiver_city','receiver_district','receiver_province']]
    
    summer_gabung.append(sumer0)

for k in range(len(hist)):
    hist0 = hist[k]
    hist0 = hist0.replace("\\", "")
    hist0 = hist0.replace('"',"")    
    hist0 = hist0.replace("'",'"')
    # hist0 = hist0.replace("'",'"')
    # hist0 = pd.read_json(hist0)
    # summer = pd.concat([summer_gabung[k]] * len(hist0), ignore_index=True)
    # hist0 = pd.concat([hist0, summer], axis = 1)

    # hist_gabung.append(hist0)
    try:
        hist0 = pd.read_json(hist0, orient='records')
        summer = pd.concat([summer_gabung[k]] * len(hist0), ignore_index=True)
        hist0 = pd.concat([hist0, summer], axis=1)
        hist_gabung.append(hist0)
        
    except ValueError:
        eror = summer_gabung[k]['waybill_number'].values[0]
        # eror_all.append(eror)
        
df = pd.concat(hist_gabung, ignore_index=True)    


In [5]:
df = df.rename(columns = 
                   {'scan_time':'Waktu Operasi',
                    'upload_time':'Waktu Pencatatan',
                    'operation_branch_name':'Diinput Cabang',
                    'operation_user_name':'Discan oleh',
                    'photo_link':'Photo Link',
                    'problem_reason':'Problem Reason',
                    'bag_number':'No. Bagging',
                    'vehicle_number':'No. Vehicle Tag',
                    'previous_location':'Lokasi Sebelumnya',
                    'next_location':'Lokasi Selanjutnya',
                    'waybill_number':'No. Waybill',
                    'order_number':'No. Order',
                    'origin_branch_name':'Origin Branch',
                    'calculated_weight':'Berat',
                    'total_shipping_fee':'Biaya Total Pengiriman',
                    'payment_type':'Tipe Pembayaran',
                    'service_type':'Tipe Servis',
                    'receiver_name':'Penerima',
                    'receiver_contact':'No.Hp penerima',
                    'receiver_province':'Provinsi Tujuan Waybill',
                    'receiver_city':'Kota Incoming',
                    'receiver_district':'Scan Tujuan',
                    'shipper_name':'Pengirim',
                    'shipper_contact':'No.HP Pengirim',


                     })



In [6]:
ts = pd.read_csv('D:/AL/DataOlahanPython/DataBase_Python/DataBantu/DataScan.csv')

In [7]:
df = pd.merge(df, ts, on = 'operation_type', how = 'left')

In [8]:
tmp = pd.read_csv('./SSR/ssrTemplateTrack.csv')

In [9]:
df = pd.concat([tmp,df],ignore_index = True)
df = df.iloc[:,:14]
df = df.sort_values('Waktu Pencatatan', ascending = False).drop_duplicates(subset = ['No. Waybill','Tipe Scan','Diinput Cabang','Waktu Pencatatan'])

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119416 entries, 12369 to 104474
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   No. Waybill              119416 non-null  object        
 1   No. Bagging              72939 non-null   object        
 2   No. Vehicle Tag          59696 non-null   object        
 3   Kota Incoming            119416 non-null  object        
 4   Tipe Scan                119416 non-null  object        
 5   Lokasi Sebelumnya        13703 non-null   object        
 6   Lokasi Selanjutnya       16253 non-null   object        
 7   Scan Tujuan              119416 non-null  object        
 8   Discan oleh              119416 non-null  object        
 9   Diinput Cabang           119416 non-null  object        
 10  Waktu Pencatatan         119416 non-null  datetime64[ns]
 11  Provinsi Tujuan Waybill  119416 non-null  object        
 12  Photo Link  

In [11]:
mitra = pd.read_csv('D:/AL/DataOlahanPython/DataBase_Python/DataBantu/TH_Sulawesi.csv')
mitra = mitra.rename(columns = {'TH Arrival':'Diinput Cabang'})
mitra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Diinput Cabang  71 non-null     object
 1   AREA            71 non-null     object
 2   District Area   71 non-null     object
 3   Kategori TH     71 non-null     object
dtypes: object(4)
memory usage: 2.3+ KB


In [12]:
df = pd.merge(df, mitra, on = 'Diinput Cabang', how = 'left')

In [13]:
df = df.iloc[:,:15]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123613 entries, 0 to 123612
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   No. Waybill              123613 non-null  object        
 1   No. Bagging              74221 non-null   object        
 2   No. Vehicle Tag          61472 non-null   object        
 3   Kota Incoming            123613 non-null  object        
 4   Tipe Scan                123613 non-null  object        
 5   Lokasi Sebelumnya        14123 non-null   object        
 6   Lokasi Selanjutnya       16705 non-null   object        
 7   Scan Tujuan              123613 non-null  object        
 8   Discan oleh              123613 non-null  object        
 9   Diinput Cabang           123613 non-null  object        
 10  Waktu Pencatatan         123613 non-null  datetime64[ns]
 11  Provinsi Tujuan Waybill  123613 non-null  object        
 12  Photo Link      

In [15]:
ssrTrack = df.copy()

In [16]:
ssrTrack.to_csv('SSR_Tracking.csv', index = False)

In [17]:
df = df[['No. Waybill','Tipe Scan','Diinput Cabang','Waktu Pencatatan','Problem Reason']]

### Olahan bg Return

In [18]:
dfal = df.copy()
dfal = dfal.drop_duplicates(subset = ['No. Waybill','Waktu Pencatatan'])

In [19]:
dfPick = df.loc[df['Tipe Scan'] == 'Pick up scan']
dfPick = dfPick.rename(columns = {'Diinput Cabang':'TH Pickup','Waktu Pencatatan':'Tgl Pickup'}).drop(columns = ['Tipe Scan','Problem Reason']).drop_duplicates()

dfConf = df.loc[df['Tipe Scan'] == 'Confirm Return Bill']
dfConf = dfConf.rename(columns = {'Waktu Pencatatan':'Tgl Konfirm'}).drop(columns = ['Tipe Scan','Diinput Cabang','Problem Reason']).drop_duplicates(subset = 'No. Waybill')

dfRt = df.loc[df['Tipe Scan'] == 'Create Return Bill']
dfRt = dfRt.rename(columns = {'Waktu Pencatatan':'Tgl regist'}).drop(columns = ['Tipe Scan','Diinput Cabang','Problem Reason']).drop_duplicates(subset = 'No. Waybill')

dfdone = df.loc[df['Tipe Scan'].isin(['POD Entry','POD scan','Return POD Scan'])]
dfdone = dfdone.rename(columns = {'Waktu Pencatatan':'Tgl Done'}).drop(columns = ['Tipe Scan','Diinput Cabang','Problem Reason']).drop_duplicates()


dfSend = df.loc[df['Tipe Scan'] == 'Sending scan']
dfSend = dfSend.loc[~dfSend['Diinput Cabang'].str[:2].isin(['MH','DC'])].sort_values('Waktu Pencatatan', ascending = True)
dfSend = dfSend.rename(columns = {'Waktu Pencatatan':'Tgl Sending'}).drop(columns = ['Tipe Scan','Diinput Cabang','Problem Reason'])


dfArv = df.loc[df['Tipe Scan'].isin(['Arrival scan','Unloading scan','Unpacking scan','Delivery scan'])]
dfArv2 = dfArv.copy()
dfArv3 = dfArv.copy()
dfArv = dfArv.loc[~dfArv['Diinput Cabang'].str[:2].isin(['MH','DC'])].sort_values('Waktu Pencatatan', ascending = True).drop_duplicates(subset = 'No. Waybill')
dfArv = dfArv.rename(columns = {'Waktu Pencatatan':'Tgl Arv'}).drop(columns = ['Tipe Scan','Diinput Cabang','Problem Reason'])

dfPos = df.loc[df['Tipe Scan'] == 'Problem On Shipment scan']
# dfPos.loc[dfPos['Problem Reason'].isin(['Paket hilang atau tidak ditemukan'])].to_csv('POS_Hilang.csv', index = False)
dfPos = dfPos.loc[~dfPos['Diinput Cabang'].str[:2].isin(['MH','DC'])]
dfalPos = dfPos.copy()
dfPos = dfPos.drop_duplicates(subset = 'No. Waybill').rename(columns = {'Waktu Pencatatan':'Tgl LastPos','Problem Reason':'Last POS'}
                                                            ).drop(columns = ['Tipe Scan','Diinput Cabang'])


dfLast = df.loc[df['Tipe Scan'] != 'Confirm Return Bill']
dfLast  = dfLast .rename(columns = {'Waktu Pencatatan':'Last Record',
                                   'Tipe Scan':'Last Scan',
                                   'Diinput Cabang':'Last Loc'}).drop(columns = ['Problem Reason']).drop_duplicates(subset = 'No. Waybill')

#### Olah LastArrv

In [20]:
dfArv3 = dfArv3.loc[~dfArv3['Diinput Cabang'].str[:2].isin(['MH','DC'])]

In [21]:
Arv = dfArv3[dfArv3['Tipe Scan'] != 'Delivery scan'].drop_duplicates(subset = 'No. Waybill')
Arv = Arv['No. Waybill']
delv = dfArv3[dfArv3['Tipe Scan'] == 'Delivery scan']

In [22]:
Arrv = pd.merge(dfal, Arv, how = 'inner')
Arrv = Arrv.loc[~Arrv['Diinput Cabang'].str[:2].isin(['MH','DC'])]

In [23]:
Arrv['No.'] = Arrv.groupby('No. Waybill')['Waktu Pencatatan'].rank(ascending = False)
Arrv['No.'] = Arrv['No.'].astype(int)
Arrv['look'] = Arrv['No. Waybill'] + Arrv['No.'].astype(str)

In [24]:
look = Arrv[['No. Waybill','Tipe Scan','Diinput Cabang','Waktu Pencatatan','No.']]
look = look.add_suffix('_2')
look['look'] = look['No. Waybill_2'] + (look['No._2']-1).astype(str)

In [25]:
Arrv = Arrv.loc[Arrv['Tipe Scan'].isin(['Arrival scan','Unloading scan','Unpacking scan'])]

In [26]:
Arrv = pd.merge(Arrv, look, on = 'look', how = 'left')

In [27]:
Arrv = Arrv.loc[(~Arrv['Tipe Scan_2'].isin(['Arrival scan','Unloading scan','Unpacking scan'])) |
                                 (Arrv['Diinput Cabang'] != Arrv['Diinput Cabang_2'])
                                  ]

In [28]:
Arrv = pd.merge(Arrv , dfRt, on = 'No. Waybill', how = 'left')

In [29]:
Arrv  = Arrv.loc[(Arrv['Tgl regist'].isna()) | (Arrv['Tgl regist'] > Arrv['Waktu Pencatatan'])]
Arrv  = Arrv[['No. Waybill','Waktu Pencatatan','Diinput Cabang']]
Arrv = Arrv.sort_values('Waktu Pencatatan', ascending = False).drop_duplicates(subset = 'No. Waybill')
Arrv = Arrv.rename(columns = {'Diinput Cabang':'LastLoc Arv', 'Waktu Pencatatan':'Last Arv'})

#### Olah FirstDelv

In [30]:
delv = pd.merge(delv, Arrv, on = 'No. Waybill', how = 'left')

In [31]:
delv = delv.loc[(delv['Last Arv'].isna()) | (delv['Last Arv'] <= delv['Waktu Pencatatan'])]

deliv_all = delv.copy()
deliv_all = deliv_all.rename(columns = {'Diinput Cabang':'TH Deliv'})

delv = delv.sort_values('Waktu Pencatatan', ascending = True).drop_duplicates(subset = 'No. Waybill')
delv = delv.drop(columns = ['Tipe Scan','Problem Reason','LastLoc Arv','Last Arv','Diinput Cabang'])
delv = delv.rename(columns = {'Waktu Pencatatan':'First Delivery'})

In [32]:
dfC = pd.merge(dfPick, Arrv, on = 'No. Waybill', how = 'left')
dfC = pd.merge(dfC, delv, on = 'No. Waybill', how = 'left')
dfC = pd.merge(dfC, dfdone, on = 'No. Waybill', how = 'left')



In [33]:
dfC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4221 entries, 0 to 4220
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   No. Waybill     4221 non-null   object        
 1   TH Pickup       4221 non-null   object        
 2   Tgl Pickup      4221 non-null   datetime64[ns]
 3   Last Arv        2792 non-null   datetime64[ns]
 4   LastLoc Arv     2792 non-null   object        
 5   First Delivery  2097 non-null   datetime64[ns]
 6   Tgl Done        214 non-null    datetime64[ns]
dtypes: datetime64[ns](4), object(3)
memory usage: 263.8+ KB


In [34]:
dfC.to_csv('Tracking_Rc.csv', index = False)

### Cek Eror

In [35]:
print(eror)

________________________
